In [1]:
import os
import os.path
import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold                                                                                                                       
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import sklearn_evaluation.plot as skplot
from sklearn.model_selection import learning_curve

import matplotlib.pyplot as plt

import numpy as np
np.random.seed(1)

from keras.preprocessing.image import img_to_array
from keras.utils import np_utils
from keras.preprocessing import image
from keras.applications.nasnet import NASNetLarge,preprocess_input

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
imagedir = "Datasets/Detection/Byteplots"

In [3]:
cur_dir = os.getcwd()
os.chdir(imagedir)  # the parent folder with sub-folders

# Get number of samples per family
list_fams = sorted(os.listdir(os.getcwd()), key=str.lower)  # vector of strings with family names
no_imgs = []  # No. of samples per family
for i in range(len(list_fams)):
    os.chdir(list_fams[i])
    len1 = len(glob.glob('*.png'))  # assuming the images are stored as 'png'
    no_imgs.append(len1)
    os.chdir('..')
num_samples = np.sum(no_imgs)  # total number of all samples

# Compute the labels
y = np.zeros(num_samples)
pos = 0
label = 0
for i in no_imgs:
    print ("Label:%2d\tFamily: %15s\tNumber of images: %d" % (label, list_fams[label], i))
    for j in range(i):
        y[pos] = label
        pos += 1
    label += 1
num_classes = label

# Compute the features
width, height,channels = (331,331,3)
X = np.zeros((num_samples, width, height, channels))
cnt = 0
list_paths = [] # List of image paths
print("Processing images ...")
for i in range(len(list_fams)):
    for img_file in glob.glob(list_fams[i]+'/*.png'):
        #print("[%d] Processing image: %s" % (cnt, img_file))
        list_paths.append(os.path.join(os.getcwd(),img_file))
        img = image.load_img(img_file, target_size=(331, 331))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        X[cnt] = x
        cnt += 1
print("Images processed: %d" %(cnt))

os.chdir(cur_dir)

Label: 0	Family:          Banker	Number of images: 1986
Label: 1	Family:          WinExe	Number of images: 1945
Processing images ...


/usr/local/lib/python3.5/dist-packages/PIL/Image.py:2438: DecompressionBombWarning: Image size (103265280 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/usr/local/lib/python3.5/dist-packages/PIL/Image.py:2438: DecompressionBombWarning: Image size (110601216 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


Images processed: 3931


In [4]:
X.shape

(3931, 331, 331, 3)

In [5]:
# Creating base_model
image_shape = (331, 331, 3)                                                                                                                                                                                                                                                                                            
base_model = NASNetLarge(weights='imagenet', input_shape=image_shape, include_top=False, pooling='avg')

In [6]:
print("Extracting bottleneck features from convolutional layers ...")
bnfeatures = base_model.predict(X)

Extracting bottleneck features from convolutional layers ...


In [7]:
bnfeatures.shape

(3931, 4032)

In [8]:
# Create stratified k-fold subsets                                                                                                                                        
kfold = 5  # no. of folds                                                                 
skf = StratifiedKFold(kfold, shuffle=True,random_state=1)

In [9]:
# Training top_model
C_range = np.logspace(-2, 10, 13)
param_grid = dict(C=C_range)

top_model = GridSearchCV(svm.LinearSVC(), param_grid=param_grid, cv=skf, verbose=10, n_jobs=-1)
top_model.fit(bnfeatures,y)  # Training 

Fitting 5 folds for each of 13 candidates, totalling 65 fits
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] ........................... C=0.01, score=0.933926, total=   1.3s
[CV] C=0.1 ...........................................................
[CV] ........................... C=0.01, score=0.940204, total=   1.3s
[CV] C=1.0 ...........................................................


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.7s


[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.949109, total=   1.4s
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.927481, total=   1.5s
[CV] C=1.0 ...........................................................
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.914758, total=   1.7s
[CV] C=10.0 ..........................................................
[CV] C=10.0 ..........................................................
[CV] C=10.0 ..........................................................
[CV] C=10.0 ..........................................................
[CV] C=10.0 ..........................................................
[CV] C=100.0 .........................................................
[CV] C=100.0 .........................................................
[CV] C

[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:   21.7s


[CV] ........................... C=10.0, score=0.941476, total=  19.5s
[CV] C=100000.0 ......................................................
[CV] ............................ C=1.0, score=0.952926, total=  20.5s
[CV] C=100000.0 ......................................................
[CV] ........................... C=10.0, score=0.936387, total=  19.9s
[CV] C=100000.0 ......................................................
[CV] ......................... C=1000.0, score=0.942748, total=  18.9s
[CV] C=1000000.0 .....................................................
[CV] ......................... C=1000.0, score=0.944091, total=  21.2s
[CV] C=1000000.0 .....................................................
[CV] ........................... C=10.0, score=0.942748, total=  24.1s
[CV] C=1000000.0 .....................................................
[CV] ............................ C=1.0, score=0.942748, total=  24.8s
[CV] C=1000000.0 .....................................................
[CV] .

...

[CV] .................... C=100000000.0, score=0.936387, total=  23.5s
[CV] .................... C=100000000.0, score=0.940280, total=  24.7s
[CV] .................... C=100000000.0, score=0.941476, total=  24.5s
[CV] ................... C=1000000000.0, score=0.933842, total=  16.0s
[CV] ................... C=1000000000.0, score=0.941476, total=  16.6s
[CV] .................. C=10000000000.0, score=0.942748, total=  15.4s
[CV] ................... C=1000000000.0, score=0.942748, total=  18.0s
[CV] ................... C=1000000000.0, score=0.936387, total=  16.6s


[Parallel(n_jobs=-1)]: Done  60 out of  65 | elapsed:   58.4s remaining:    4.9s


[CV] .................. C=10000000000.0, score=0.933842, total=  14.5s
[CV] .................. C=10000000000.0, score=0.941476, total=  16.1s
[CV] .................. C=10000000000.0, score=0.932570, total=  14.4s
[CV] .................. C=10000000000.0, score=0.940280, total=  17.4s


[Parallel(n_jobs=-1)]: Done  65 out of  65 | elapsed:   59.9s finished


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=1, shuffle=True),
       error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05,
       1.e+06, 1.e+07, 1.e+08, 1.e+09, 1.e+10])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=10)

In [10]:
top_model.cv_results_

{'mean_fit_time': array([ 1.36975579, 11.08852572, 23.16150756, 20.03606768, 25.10354853,
        22.06618133, 21.88596878, 19.85658188, 22.92552934, 23.67585301,
        22.95639009, 17.28919611, 15.53213077]),
 'mean_score_time': array([0.05247679, 0.09488249, 0.07989583, 0.05788269, 0.09817872,
        0.07317896, 0.05329208, 0.06567254, 0.04733424, 0.05387931,
        0.01812425, 0.03296547, 0.02446089]),
 'mean_test_score': array([0.9330959 , 0.94734164, 0.94556093, 0.93945561, 0.93843806,
        0.93869244, 0.93818367, 0.93869244, 0.93894683, 0.93920122,
        0.93894683, 0.93894683, 0.93818367]),
 'mean_train_score': array([0.94460697, 0.97468833, 0.99847365, 1.        , 0.99993641,
        0.99993641, 0.99993641, 0.99993641, 0.99993641, 0.99993641,
        0.99993641, 0.99993641, 0.99993641]),
 'param_C': masked_array(data=[0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0,
                    1000000.0, 10000000.0, 100000000.0, 1000000000.0,
                    1000000

In [11]:
top_model.grid_scores_

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.93310, std: 0.01162, params: {'C': 0.01},
 mean: 0.94734, std: 0.01005, params: {'C': 0.1},
 mean: 0.94556, std: 0.00444, params: {'C': 1.0},
 mean: 0.93946, std: 0.00367, params: {'C': 10.0},
 mean: 0.93844, std: 0.00431, params: {'C': 100.0},
 mean: 0.93869, std: 0.00438, params: {'C': 1000.0},
 mean: 0.93818, std: 0.00366, params: {'C': 10000.0},
 mean: 0.93869, std: 0.00295, params: {'C': 100000.0},
 mean: 0.93895, std: 0.00322, params: {'C': 1000000.0},
 mean: 0.93920, std: 0.00295, params: {'C': 10000000.0},
 mean: 0.93895, std: 0.00332, params: {'C': 100000000.0},
 mean: 0.93895, std: 0.00332, params: {'C': 1000000000.0},
 mean: 0.93818, std: 0.00416, params: {'C': 10000000000.0}]

In [12]:
print("Best parameters: %s - score: %0.4f" % (top_model.best_params_, top_model.best_score_))

Best parameters: {'C': 0.1} - score: 0.9473
